# Template for TSP Algorithm in Julia

Solves the Symmetric TSP using a [insert]

# Add packages and imports

In [1]:
# import Pkg
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [2]:
using DataFrames
using CSV
using LinearAlgebra
using Random

# Load data from csv

This is the 70 city problem from TSP lib.

In [3]:
st70 = CSV.read("data/st70.csv");

In [4]:
head(st70, 10)

,city,x,y
,Int64,Int64,Int64
1,1,64,96
2,2,80,39
3,3,69,23
4,4,72,42
5,5,48,67
6,6,58,43
7,7,81,34
8,8,79,17
9,9,30,23


# Functions

* euclidean_distance_matrix - compute cost matrix
* trim_cities - make problem instance smaller than 70 cities!
* tour_cost - compute cost of tour
* tweak! - randomly swap two cities in tour
* hill_climb - find a good solution within a given time limit.

In [5]:
"""
    euclidean_distance_matrix(cities)

Compute a matrix of euclidean distances between
city x, y coordinate pairs

# Arguments
- cities::Array: n x 2 matrix of x, y coordinates

# Examples
```julia-repl 
julia> using Random;

julia> Random.seed!(42);

julia> coords = rand(1:50, 5, 2)
5×2 Array{Int64,2}:
 16  42
 10  15
 44  36
 32  42
 28  39

julia> euclidean_distance_matrix(coords)
5×5 Array{Float64,2}:
  0.0     27.6586  28.6356  16.0     12.3693
 27.6586   0.0     39.9625  34.8281  30.0
 28.6356  39.9625   0.0     13.4164  16.2788
 16.0     34.8281  13.4164   0.0      5.0
 12.3693  30.0     16.2788   5.0      0.0
```
"""
function euclidean_distance_matrix(cities)
    nrows = size(cities)[1]
    matrix = zeros(nrows, nrows)
    
    row = 1
    
    for city1 in 1:nrows
        col = 1
        for city2 in 1:nrows
            matrix[row, col] = norm(cities[city1, 1:2]-cities[city2, 1:2])
            col+=1
        end
        row +=1
    end
        
    return matrix
end

euclidean_distance_matrix

In [6]:
trim_cities(df, ncities) = Array(df[2:end])[1:ncities, :]

trim_cities (generic function with 1 method)

In [7]:
"""
    tour_cost(tour, matrix)

Compute the travel cost of tour using
the cost matrix
"""
function tour_cost(tour, matrix)
    cost = 0.0
    for i in 1:size(tour)[1] - 1
        cost += matrix[tour[i], tour[i+1]]
    end
    
    cost += matrix[tour[end], tour[1]]
    
    return cost
end

tour_cost

In [8]:
"""
    simple_tweak(tour)

Randomly select to elements within the 
vector tour and swap them
"""
function simple_tweak(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour[sample[1]], tour[sample[2]] = tour[sample[2]], tour[sample[1]]
    return tour
end

simple_tweak

In [9]:
function tweak_two_opt(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour = reverse(tour, tour[sample[1]], tour[sample[2]])
    return tour
end

tweak_two_opt (generic function with 1 method)

# Algorithm

In [14]:
function random_restarts(initial_solution, matrix; time_limit=2)
    best = copy(initial_solution)
    best_cost = -tour_cost(initial_solution, matrix)
    
    start = time()
    while (time() - start) < time_limit
        neighbour = shuffle(copy(initial_solution))
        neighbour_cost = -tour_cost(neighbour, matrix)
        
        if neighbour_cost > best_cost
            best, best_cost = neighbour, neighbour_cost
        end
    end
    
    return -best_cost, best
end 

random_restarts (generic function with 1 method)

# Example solution

In [ ]:
Random.seed!(42);

coords = trim_cities(st70, 20);
matrix = euclidean_distance_matrix(coords)
tour = [i for i in 1:size(coords)[1]]

random_restarts(tour, matrix, time_limit=10)